# SETUP

## is GPU on?

In [1]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

## boilerplate

In [1]:
from tensorflow.keras.callbacks import Callback, EarlyStopping
from numpy import arange

MODEL_NAME = 'mobilenet'
IMG_SIZE = (224, 224)
INPUT_SHAPE=(224, 224, 3)
CLASSES = 2
TRIAL = list(arange(3,10)) #list(arange(0,3))
FT_BLOCK = list(arange(0,11)) # FROM feature extractor TO fine tuning scratch
BATCH_SIZE = [32, 64, 128, 256] # these data points will be passed as a batch at one time to the network

## UDC/Fs

In [2]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras import layers
from math import floor
from timeit import default_timer as timer
from json import dump
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import data
from keras.engine import training

def model_constructor(FT_BLOCK):
    base_model = MobileNet(
        weights='imagenet', 
        include_top=False,
        input_shape=INPUT_SHAPE)
    
    # construct the top layer containing 
    x = base_model.output
    x = layers.GlobalAveragePooling2D(keepdims=True)(x)
    x = layers.Dropout(1e-3, name='dropout')(x)
    x = layers.Conv2D(CLASSES, (1, 1), padding='same', name='conv_preds')(x)
    x = layers.Reshape((CLASSES,), name='reshape_2')(x)
    x = layers.Activation(activation='sigmoid', name='predictions')(x)
    
    # model is ready to be trained
    alpha, rows = 1.0, 224
    model = training.Model(inputs=base_model.input, outputs=x, name='mobilenet_%0.2f_%s' % (alpha, rows))

    # freeze the layers before the `FROZEN_END` layer 
    total_layers = len(base_model.layers)
    ft_layers = floor(total_layers / 10)
    FROZEN_END = total_layers - ft_layers * FT_BLOCK

    base_model.trainable = True
    for layer in base_model.layers[:FROZEN_END]: 
        layer.trainable = False
    
    return model

class TimeCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

def time_converter(sec):
    hours, rem = divmod(sec, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

def save_history(history, tag):
    file_path = f'{HISTORY_DIR}/{tag}.json'
    with open(file_path, 'w') as f:
        dump(history.history, f)
        
def data_preparation(BATCH_SIZE):
    train_dir = '../../data/binaryclass_clean/train/'
    test_dir = '../../data/binaryclass_clean/test/'

    train_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='training')

    val_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='validation')

    test_ds = image_dataset_from_directory(
        directory=test_dir,
        label_mode='categorical',
        batch_size=1,
        image_size=IMG_SIZE)

    AUTOTUNE = data.AUTOTUNE
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

# GRID SEARCH

In [4]:
for i in TRIAL:
    print(" - - - - - TRIAL:", i, " - - - - - ")
    HISTORY_DIR = f'../../logs/FT/{i}'
    for batch in BATCH_SIZE:
        print("Batch size:", batch)
        train_ds, val_ds, test_ds = data_preparation(batch)
        epochs, loss, accuracy = [None]*len(FT_BLOCK), [None]*len(FT_BLOCK), [None]*len(FT_BLOCK)
        for ft in FT_BLOCK:
            tag = f'{MODEL_NAME}_BS{batch}_FT{ft*10}'
            print(tag)
            model = model_constructor(ft)
            model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')
            history = model.fit(train_ds, epochs=100, verbose=0, validation_data=val_ds, callbacks=[EarlyStopping(patience=3)])
            save_history(history, tag)
            # time_converter(sum(cb_time.logs))
            epochs[ft] = len(history.history['loss'])
            loss[ft], accuracy[ft] = model.evaluate(test_ds)
        # PRINT THE RESULT
        # print('Count Epoch:', epochs)
        print(f' * * * * * {MODEL_NAME}_BS{batch}_LOSS * * * * * ')
        for item in loss:
            print(item)
        print(f' * * * * * {MODEL_NAME}_BS{batch}_ACCURACY * * * * * ')
        for item in accuracy:
            print(item)

 - - - - - TRIAL: 0  - - - - - 
Batch size: 32
Found 9980 files belonging to 2 classes.
Using 8982 files for training.
Found 9980 files belonging to 2 classes.
Using 998 files for validation.
Found 1000 files belonging to 2 classes.
mobilenet_BS32_FT0
1000/1000 [==============================] - 9s 9ms/step - loss: 0.5913 - accuracy: 0.6840
mobilenet_BS32_FT10
1000/1000 [==============================] - 18s 18ms/step - loss: 2.2249 - accuracy: 0.4980
mobilenet_BS32_FT20
1000/1000 [==============================] - 11s 11ms/step - loss: 4.0339 - accuracy: 0.5000
mobilenet_BS32_FT30
1000/1000 [==============================] - 16s 16ms/step - loss: 0.7060 - accuracy: 0.6930
mobilenet_BS32_FT40
1000/1000 [==============================] - 17s 17ms/step - loss: 0.8103 - accuracy: 0.6610
mobilenet_BS32_FT50
1000/1000 [==============================] - 16s 16ms/step - loss: 0.7866 - accuracy: 0.7730
mobilenet_BS32_FT60
1000/1000 [==============================] - 18s 17ms/step - loss: 0.706

1000/1000 [==============================] - 21s 21ms/step - loss: 0.9109 - accuracy: 0.7070
mobilenet_BS32_FT50
1000/1000 [==============================] - 14s 14ms/step - loss: 0.8386 - accuracy: 0.7180
mobilenet_BS32_FT60
1000/1000 [==============================] - 15s 15ms/step - loss: 0.8455 - accuracy: 0.6420
mobilenet_BS32_FT70
1000/1000 [==============================] - 11s 11ms/step - loss: 0.6626 - accuracy: 0.8300
mobilenet_BS32_FT80
1000/1000 [==============================] - 13s 13ms/step - loss: 0.7384 - accuracy: 0.8150
mobilenet_BS32_FT90
1000/1000 [==============================] - 11s 11ms/step - loss: 0.4882 - accuracy: 0.8630
mobilenet_BS32_FT100
1000/1000 [==============================] - 16s 16ms/step - loss: 0.4694 - accuracy: 0.8590
 * * * * * mobilenet_BS32_LOSS * * * * * 
0.5887998938560486
2.207819938659668
2.4994165897369385
1.8698177337646484
0.9108635783195496
0.8385953307151794
0.8454824686050415
0.6626448631286621
0.7384312748908997
0.48815417289733